# 13. Parallelizing Neural Netowrk Training with TensorFlow

In [1]:
%run -i 'watermark.py'

2020-07-23 20:17:49
----------------------
python		3.6.7
----------------------
numpy		1.16.2
scipy		1.1.0
pandas		0.25.1
matplotlib	3.1.1
imageio		2.5.0
----------------------
ipython		7.8.0
----------------------
sklearn		0.20.4
tensorflow	1.13.1
nltk		3.2.4
----------------------
networkx	2.0


## 13.01. TensorFlow and training performance

### 13.01.01. Performance challenges

### 13.01.02. What is TensorFlow?

![Fig.13.1](https://raw.githubusercontent.com/rasbt/python-machine-learning-book-3rd-edition/master/ch13/images/01.png)

### 13.01.03. How we will learn TensorFlow

## 13.02. First steps with TensorFlow

### 13.02.01. Installing TensorFlow

### 13.02.02. Creating ttensors in TensorFlow



In [2]:
import tensorflow as tf
import numpy as np

np.set_printoptions(precision=3)
a = np.array([1, 2, 3], dtype=np.int32)
b = [4, 5, 6]
t_a = tf.convert_to_tensor(a)
t_b = tf.convert_to_tensor(b)
print(t_a)
print(t_b)

Tensor("Const:0", shape=(3,), dtype=int32)
Tensor("Const_1:0", shape=(3,), dtype=int32)


In [3]:
t_ones = tf.ones((2, 3))
t_ones.shape

TensorShape([Dimension(2), Dimension(3)])

In [4]:
t_ones.numpy()

AttributeError: 'Tensor' object has no attribute 'numpy'

In [5]:
const_tensor = tf.constant([1.2, 5, np.pi], dtype=tf.float32)
print(const_tensor)

Tensor("Const_2:0", shape=(3,), dtype=float32)


## 13.02.03. Manipulating the data type and shape of a tensor

`tf.cast()`

In [7]:
t_a_new = tf.cast(t_a, tf.int64)
print(t_a_new.dtype)

<dtype: 'int64'>


`tf.transpose()`, `tf.reshape()`, `tf.squeeze()`

In [9]:
t = tf.random.uniform(shape=(3, 5))
t_tr = tf.transpose(t)
print(t.shape, '-->', t_tr.shape)

(3, 5) --> (5, 3)


In [10]:
t = tf.zeros((30,))
t_reshape = tf.reshape(t, shape=(5, 6))
print(t_reshape.shape)

(5, 6)


In [11]:
t = tf.zeros((1, 2, 1, 4, 1))
t_sqz = tf.squeeze(t, axis=(2,4))
print(t.shape, '-->', t_sqz.shape)

(1, 2, 1, 4, 1) --> (1, 2, 4)


### 13.02.04. Applying matheatical operations to tensors 

In [16]:
# tf.random.set_seed(1)
t1 = tf.random.uniform(shape=(5,2), minval=1.0, maxval=1.0)
t2 = tf.random.normal(shape=(5,2), mean=0.0, stddev=1.0)

In [17]:
t3 = tf.multiply(t1, t2).numpy()
print(t3)

AttributeError: 'Tensor' object has no attribute 'numpy'

`tf.math.reduce_mean()`, `tf.math.reduce_sum()`, `tf.math.reduce_std()`

In [18]:
t4 = tf.math.reduce_mean(t1, axis=0)
print(t4)

Tensor("Mean:0", shape=(2,), dtype=float32)


\( t_1 \times t_2^T \)

`tf.linalg.matmul()`

In [19]:
t5 = tf.linalg.matmul(t1, t2, transpose_b=True)
print(t5.numpy())

AttributeError: 'Tensor' object has no attribute 'numpy'

In [ ]:
t6 = tf.linalg.matmul(t1, t2, transpose_a=True)
print(t6.numpy())